# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kingsland,30.8000,-81.6898,27.74,91,26,1.59,US,1689836931
1,1,adamstown,-25.0660,-130.1015,22.66,78,100,12.23,PN,1689836980
2,2,port saint john's,-31.6229,29.5448,13.35,68,73,6.62,ZA,1689836980
3,3,invercargill,-46.4000,168.3500,4.47,78,4,4.57,NZ,1689836980
4,4,ribeira grande,38.5167,-28.7000,20.43,94,75,3.60,PT,1689836981


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": city_data_df["Lat"],
    "longitude": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}
coordinates_df = pd.DataFrame(coordinates)

map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    scale=1,
    color="City"
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] < 27), :]

# Drop any rows with null values
ideal_city.dropna

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,adamstown,-25.0660,-130.1015,22.66,78,100,12.23,PN,1689836980
8,8,calabozo,8.9242,-67.4293,22.90,96,100,0.51,VE,1689836981
11,11,charters towers,-20.1000,146.2667,24.59,34,33,2.48,AU,1689836982
13,13,karratha,-20.7377,116.8463,26.72,28,84,4.61,AU,1689836983
14,14,xilin hot,43.9667,116.0333,26.53,46,100,1.78,CN,1689836983
...,...,...,...,...,...,...,...,...,...,...
570,570,touba,8.2833,-7.6833,21.05,98,100,2.71,CI,1689837129
571,571,srivardhan,18.0333,73.0167,26.51,88,100,6.01,IN,1689837130
573,573,mirny,62.5353,113.9611,25.13,36,29,4.92,RU,1689837130
576,576,melioratorov,57.2107,65.6077,24.24,57,0,3.00,RU,1689837131


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City", "Country", "Lat", "Lng", "Country", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Country,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,PN,78,
8,calabozo,VE,8.9242,-67.4293,VE,96,
11,charters towers,AU,-20.1000,146.2667,AU,34,
13,karratha,AU,-20.7377,116.8463,AU,28,
14,xilin hot,CN,43.9667,116.0333,CN,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
calabozo - nearest hotel: Gran Hotel Guarico
charters towers - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
xilin hot - nearest hotel: Hotel hailibu 120
carutapera - nearest hotel: No hotel found
bayganin - nearest hotel: No hotel found
daigo - nearest hotel: 豊年万作
newman - nearest hotel: No hotel found
andovoranto - nearest hotel: Le nirvana
alice springs - nearest hotel: Aurora Alice Springs
talara - nearest hotel: Hospedaje El Aypate
udachny - nearest hotel: Вилюй
hamilton - nearest hotel: North Vista Manor
kindu - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
gurupa - nearest hotel: No hotel found
yerofey pavlovich - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
cabedelo - nearest hotel: No hotel found
labytnangi - nearest hotel: Семь лиственниц
larache - nearest hotel: Hotel Tanger
puerto ayora - nearest hotel: Hostal La

,City,Country,Lat,Lng,Country,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,PN,78,No hotel found
8,calabozo,VE,8.9242,-67.4293,VE,96,Gran Hotel Guarico
11,charters towers,AU,-20.1000,146.2667,AU,34,No hotel found
13,karratha,AU,-20.7377,116.8463,AU,28,Karratha International Hotel
14,xilin hot,CN,43.9667,116.0333,CN,46,Hotel hailibu 120
...,...,...,...,...,...,...,...
570,touba,CI,8.2833,-7.6833,CI,98,No hotel found
571,srivardhan,IN,18.0333,73.0167,IN,88,No hotel found
573,mirny,RU,62.5353,113.9611,RU,36,Зарница
576,melioratorov,RU,57.2107,65.6077,RU,57,Юрга


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": hotel_df["Lat"],
    "longitude": hotel_df["Lng"],
    "City": hotel_df["City"],
    "Humidity": hotel_df["Humidity"],
    "Hotel": hotel_df["Hotel Name"]
}
coordinates_df = pd.DataFrame(coordinates)

map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["Hotel"]
).opts(tools=['hover'])


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,Hotel)